Fixed length FIFO cache using Redis

Useful for streaming applications, where data keeps coming in.
REDIS cache will store the objects streaming in to server - API based on REDIS can now be used to retrieve latest top x enteries in DB.

In [10]:
import redis
class redisFIFOQ:
    
    newsQueueName = "news"
    
    #Create Redis sliding window object, default size 100
    def __init__(self, queueSize = 100, host="localhost", port=6379):
        self.db = redis.Redis(host=host, port=port)
        self.newsQueueSize = queueSize
    
    def put(self, key, value):
        if not isinstance(value,dict):
            print ("Value needs to be a dictionary " + key)
            return False
        if not isinstance(key,str):
            print ("Key needs to be a string " + key)
            return False
        self.db.hmset(key, value)
        return True
    
    def get(self, key):
        if not isinstance(key,str):
            print ("Key needs to be a string " + key)
            return False
        elif not self.db.exists(key):
            print ("Key doesnt exist " + key)
            return False
        
        val = self.db.hgetall(key)
        return val
    
    def delete(self, key):
        if key == 'news':
            print ("Call freeQueue() function instead")
            return False
        
        if not self.db.exists(key):
            print ("Key doesnt exist " + key)
            return False
        
        self.db.delete(key)
        return True
    
    def deleteAll(self):
        for key in self.db.keys():
            # delete the key
            self.db.delete(key)
                      
    def checkIfKeyExists(self, key):
        if self.db.exists(key):
            return True
        else:
            return False
        
    ######################Queue implementation starts here###################
    
    
    def getLengthOfQueue(self):
        return self.db.llen(self.newsQueueName)
    
    #Check if Queue full
    def isQueueFull(self):
        if self.getLengthOfQueue() <= self.newsQueueSize:
            return False
        return True
    
    #pushed value from end of queue (Pops oldest news)
    def popQ(self):
        val = self.db.rpop(self.newsQueueName)
        return val 
    
    #pushed value to top of news queue
    #Returns True if queue is over the capacity, else false
    def pushQ(self, value): 
        #Dont push if object corresponding to key being pushed into queue exists
        if self.checkIfKeyExists(value):
            print ("This key already exists")
            return False
        self.db.lpush(self.newsQueueName,value)
        if self.isQueueFull():
            return 2
        return True
    
    #Get values from start to end positions of newws queue
    def getFromQueue(self, start, end):
        val = self.db.lrange(self.newsQueueName, start, end)
        return val
    
    #Get entire news queue content
    def getQ(self):
        return self.getFromQueue(0,self.newsQueueSize-1)
    
    def freeQueue(self):
        keys = self.getQ()
        for key in keys:
            key = key.decode("utf-8")
            self.delete(key)
        self.db.delete('news')
    
    ################Sliding window key-value object storage implementation##################
    
    #Insert into sliding window
    def insertObject(self, key, value):
        key = str(key)
        print("Pushing key ", key)
        
        if key == self.newsQueueName:
            print ("Cant use this key")
            return False
        
        isQueueFull = self.pushQ(key)
        if not isQueueFull:
            print ("Something went wrong")
            return False
        
        if isQueueFull == 2:
            keyToDelete = self.popQ()
            self.delete(keyToDelete)
            
        self.put(key, value)
        return True
    
    #Get sliding windows content
    def getObjects(self):
        keys = self.getQ()
        objects = []
        for key in keys:
            key = key.decode("utf-8") 
            value = self.get(key)
            dct = {key:value}
            objects.append(dct)
        
        return objects
            
            

In [11]:
#host="daix-news-api.fcryea.ng.0001.use1.cache.amazonaws.com", port=6379

In [12]:
#Initiate Redis sliding windows object. Pass queue size as param. Default size = 100
db = redisFIFOQ(5)

In [13]:
#REDIS Key value storage functions
db.checkIfKeyExists('Jay')
db.put("Jay", {"surname" : "jaylohokare"})
db.get("Jay")
db.delete("Jay")


ConnectionError: Error 10061 connecting to localhost:6379. No connection could be made because the target machine actively refused it.

In [ ]:
#Sliding window insert (key, dictionary). Key has to be unique for all objects
db.insertObject("8", {"val" :"One"})

In [ ]:
#Get sliding window content
db.getObjects()

In [398]:
#Get queue content -> Get all keys in sliding window
db.getQ()

[b'8', b'7']

In [389]:
#Delete the sliding window content
db.freeQueue()

Key doesnt exist news


False

In [ ]:
#Use this function cautiously! Deletes REDIS content
db.deleteAll()